In [1]:
import pandas as pd
from sklearn import preprocessing
import argparse
import numpy as np
import scipy
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from time import time
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.grid_search import ParameterGrid
import pylab as pl
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import ParameterGrid
from scipy import interp
from sklearn.cross_validation import StratifiedKFold

In [2]:
data = pd.read_csv('bank-additional-full.csv', delimiter=';')

# Label Encoding
lab = preprocessing.LabelEncoder()
data.job = lab.fit_transform(data.job)
data.marital = lab.fit_transform(data.marital)
data.education = lab.fit_transform(data.education)
data.default = lab.fit_transform(data.default)
data.housing = lab.fit_transform(data.housing)
data.loan = lab.fit_transform(data.loan)
data.contact = lab.fit_transform(data.contact)
data.month = lab.fit_transform(data.month)
data['day_of_week'] = lab.fit_transform(data['day_of_week'])
data.poutcome = lab.fit_transform(data.poutcome)
data.y = lab.fit_transform(data.y)

In [3]:
features = data.as_matrix() # features shape: (41188, 21)
target = features[:, 20]
features_numeric = features[:, 0:20] # shape: (41188, 20) (index 20 won't be included)

In [4]:
print(features_numeric[0])
# [  5.60000000e+01   3.00000000e+00   1.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
#    6.00000000e+00   1.00000000e+00   2.61000000e+02   1.00000000e+00
#    9.99000000e+02   0.00000000e+00   1.00000000e+00   1.10000000e+00
#    9.39940000e+01  -3.64000000e+01   4.85700000e+00   5.19100000e+03]

# One-k-scheme encoding
from sklearn.preprocessing import OneHotEncoder
cat_indices = np.array([[1,2,3,4,5,6,7,8,9,14]]) # [[ 1  2  3  4  5  6  7  8  9 14]]

enc = OneHotEncoder(categorical_features = cat_indices)
encoded_features1 = enc.fit_transform(features_numeric)
print(enc.n_values_)
# [12  4  8  3  3  3  2 10  5  3]

print(enc.feature_indices_)
# [ 0 12 16 24 27 30 33 35 45 50 53]

print(len(encoded_features1.toarray()[0]))
# 63

print(encoded_features1.toarray()[0]) # print the first row of data (it has rearranged category features in front of all continuous features)
# [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00(housemaid, which is encoded as 3)
#    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
#    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
#    1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
#    1.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
#    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
#    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
#    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
#    0.00000000e+00   5.60000000e+01   2.61000000e+02   1.00000000e+00
#    9.99000000e+02   0.00000000e+00   1.10000000e+00   9.39940000e+01
#   -3.64000000e+01   4.85700000e+00   5.19100000e+03]


[  5.60000000e+01   3.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
   6.00000000e+00   1.00000000e+00   2.61000000e+02   1.00000000e+00
   9.99000000e+02   0.00000000e+00   1.00000000e+00   1.10000000e+00
   9.39940000e+01  -3.64000000e+01   4.85700000e+00   5.19100000e+03]
[12  4  8  3  3  3  2 10  5  3]
[ 0 12 16 24 27 30 33 35 45 50 53]
63
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
   0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+

In [5]:
# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(encoded_features1.todense(), target, test_size=0.4, random_state=0)
print(X_train.shape)
print(X_test.shape)

(24712, 63)
(16476, 63)


In [6]:
def run_classifier(classifier_id):    
    classifiers = (None, RandomForestClassifier, SVC, LogisticRegression, DecisionTreeClassifier)

    # grid1 = [{'n_estimators' : [50, 100, 250, 500, 1000]}]
    grid1 = [{'n_estimators' : [50, 100, 250]}]
    grid2 = [{'kernel': ['rbf', 'linear'], 'C': [0.1, 1.0, 100.0],'gamma': [0.001, 0.1, 10.0], 'class_weight' : ['auto'], 'probability' : [True]}]
    grid3 = [{'C' : [1], 'class_weight' : ['auto']}]
    grid4 = [{'max_features' : ['sqrt', 'log2', 20]}]

    grids = (None, grid1, grid2, grid3, grid4)
    grid_obj = grids[classifier_id]
    cls_obj = classifiers[classifier_id]

    best_score = None
    print ("Performing cross validation using parameter grid...")
    for one_param in ParameterGrid(grid_obj):
        cls = cls_obj(**one_param)

        one_score = cross_val_score(cls, X_train, Y_train, cv=5, scoring = 'roc_auc')
        mscore = one_score.mean()

        print ("param=%s, score=%.6f" % (repr(one_param),mscore))

        if ( best_score is None or best_score < mscore): 
            best_param = one_param
            best_score = mscore
            best_svc = cls

    print ("Best score for Cross Validation: %.6f" % best_score)

    ### ROC
    roc_name = (None, "Random Forest", "SVM", "Logistic Regression", "Decision Trees")
    probas_ = best_svc.fit(X_train, Y_train).predict_proba(X_test)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(Y_test, probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    print("Area under the ROC curve : %f" % roc_auc)

    # Plot ROC curve
    pl.clf()
    pl.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    pl.plot([0, 1], [0, 1], 'k--')
    pl.xlim([0.0, 1.0])
    pl.ylim([0.0, 1.0])
    pl.xlabel('False Positive Rate')
    pl.ylabel('True Positive Rate')
    pl.title('Receiver operating characteristic- %s' % (roc_name[classifier_id]))
    pl.legend(loc="lower right")
    pl.show()

In [7]:
run_classifier(1)

Performing cross validation using parameter grid...
param={'n_estimators': 50}, score=0.933130
param={'n_estimators': 100}, score=0.934542
param={'n_estimators': 250}, score=0.937967
Best score for Cross Validation: 0.937967
Area under the ROC curve : 0.944741


In [ ]:
# TODO BUG: cannot run SVM
run_classifier(2)

In [7]:
run_classifier(3)

/Users/de-weikung/anaconda/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/de-weikung/anaconda/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/de-weikung/anaconda/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/de-weikung/anaconda/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 

Performing cross validation using parameter grid...
param={'C': 1, 'class_weight': 'auto'}, score=0.935833

/Users/de-weikung/anaconda/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
/Users/de-weikung/anaconda/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)



Best score for Cross Validation: 0.935833
Area under the ROC curve : 0.938485


In [7]:
run_classifier(4)

Performing cross validation using parameter grid...
param={'max_features': 'sqrt'}, score=0.706928
param={'max_features': 'log2'}, score=0.702946
param={'max_features': 20}, score=0.721002
Best score for Cross Validation: 0.721002
Area under the ROC curve : 0.733880
